In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import collections

%matplotlib inline
plt.style.use('ggplot')

In [2]:
filename = '219-CATTRACKS Fall 2019- February Billing.xlsx'

xlsx_df = pd.ExcelFile(filename)
xlsx_df.sheet_names

['CAT BILL',
 'FARES',
 'DELAYS',
 'BIKES',
 'C1 BLUE',
 'C1-GOLD',
 'C2 - EXPRESS',
 'SAT-SUN E1-LINE DOWNTOWN',
 'E1 HOLIDAY',
 'SAT-SUN E2-LINE HOUSING',
 'E2 HOLIDAY',
 'Fastcat',
 'G-Line',
 'Heritage 1 ',
 'Heritage 2',
 'SAT-SUND HERITAGE',
 'FR-SAT NITECAT',
 'PAX LEFT',
 'STANDBY',
 'AB LINE',
 'E-Line']


Based on assessment of csv data, most informative sheets in workbook consist of these bus lines:
 'C1 BLUE',
 'C1-GOLD',
 'C2 - EXPRESS',
 'G-Line'

Will only read those sheets into a pandas df
And then into a dictionary to track of the bus line and its total load data


In [3]:
use_sheets = ['C1 BLUE', 'C1-GOLD', 'C2 - EXPRESS', 'G-Line']
df_names = ['c1_blue', 'c1_gold', 'c2_exp', 'g_line']

df_dicts = {}
for i in range(len(df_names)):
    df_dicts[df_names[i]] = pd.read_excel(filename, use_sheets[i])

In [4]:
# Hate that I am doing this brute force, but didn't know how to grab these totals otherwise:
# Want an ordered dictionary since I am using brute means to slice data

collections.OrderedDict(df_dicts)
df_dicts.keys()


# Slices obtained through visual analysis and trial and error. Will not be able to 
# get away with doing this on a larger data set

c1_blue = df_dicts['c1_blue'].iloc[4:25, 86:101]
c1_gold = df_dicts['c1_gold'].iloc[4:25, 81:95]
c2_exp = df_dicts['c2_exp'].iloc[4:19, 106:125]
g_line = df_dicts['g_line'].iloc[4:17, 101:119]

# Step above removed use of dictionaries. Just dfs now

print(type(g_line))

<class 'pandas.core.frame.DataFrame'>


In [5]:
# Only c1_blue and c1_gold lines have Nans in first column.
# c2_exp, g_line do not have Nans in first column

nans_lst = [c1_blue, c1_gold]

for df in nans_lst:
    df.dropna(inplace=True)
    print(df)

               Unnamed: 86  Unnamed: 87 Unnamed: 88  Unnamed: 89  Unnamed: 90  \
4            C1 Blue -Line        545.0         704        843.0       1002.0   
5           Granville Apts          0.0          31         61.0         41.0   
6                  Walmart          0.0           0          1.0          0.0   
7        Meadows/Olivewood          0.0          29         39.0         29.0   
8       Alexander & "G" St          2.0          24         50.0         38.0   
9        Rite Aid/Walgreen          2.0          19         84.0         31.0   
10         El Portal Plaza          0.0          21         42.0         37.0   
11          Mercy Hospital          3.0           0          0.0          2.0   
12           Arrow Wood Dr         11.0         194        303.0        116.0   
13               Muir Pass          5.0           2         85.0        124.0   
14       Scholars/Emigrant          1.0           1          7.0         13.0   
15   TriCol/Mercy Hospital  

In [6]:
# These two columns did not contain any data. 
# Not sure why they are there to begin with. Removed them.

c2_exp = c2_exp.drop(columns=['Unnamed: 123'])
g_line = g_line.drop(columns=['Unnamed: 117'])

# print(c2_exp), print(g_line)

In [7]:
# Check that these are the columns to plot in a bar graph

print(c1_blue.iloc[1:, 0])
c1_blue.iloc[1:, -1]

5             Granville Apts
6                    Walmart
7          Meadows/Olivewood
8         Alexander & "G" St
9          Rite Aid/Walgreen
10           El Portal Plaza
11            Mercy Hospital
12             Arrow Wood Dr
13                 Muir Pass
14         Scholars/Emigrant
15     TriCol/Mercy Hospital
16    El Portal Plaza/"G" St
17        "G" St & Alexander
18         Meadows/Olivewood
19                   Walmart
20            Granville Apts
24                     TOTAL
Name: Unnamed: 86, dtype: object


5      251
6       27
7      160
8      220
9      253
10     210
11      19
12     937
13    1893
14     223
15       7
16      16
17      32
18      18
19      38
20       0
24    4304
Name: Unnamed: 100, dtype: object

In [ ]:
dfs_lst = [c1_blue, c1_gold, c2_exp, g_line]
c1_blue.iloc[1:, 0]
c1_blue.iloc[1:, -1]

fig, axs = plt.subplots(2,2, figsize = (18,14), sharey=True, constrained_layout=True)
for i, ax in enumerate(axs.flat):
    ax.bar(dfs_lst[i].iloc[1:, 0], dfs_lst[i].iloc[1:, -1])
    ax.set_xticklabels(labels=dfs_lst[i].iloc[1:, 0], rotation=45, ha='right')
    ax.set_title(dfs_lst[i].iloc[0,0])
    ax.set_ylabel('# Students')

fig.suptitle('UCM CatTracks - Feb 2019 Bus Load Data', fontsize=24)
# fig.savefig('2019-feb-plot.png')

Text(0.5, 0.98, 'UCM CatTracks - Feb 2019 Bus Load Data')

In [ ]:
c1_blue

In [ ]:
c1_gold

In [ ]:
c2_exp

In [ ]:
g_line